# Module 5: Assignment - Comparison Exercise - Analyzing Deep Learning Frameworks

In [2]:
%pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 25.7 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.6-py3-none-any.whl size=111949 sha256=077829b5bd11439df551dafe414d894bf69f8a0e8e21dcefaa01af6008fe1669
  Stored in directory: /root/.cache/pip/wheels/0a/e0/6d/fbc9dff0a755904e8a1a5e0730d0918114270dc17c391c9a91
Successfully built kaggle

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Download dataset
dataset_name = 'gpiosenka/100-bird-species'
path_to_download = 'birds'  # Specify your download path here

api.dataset_download_files(dataset_name, path=path_to_download, unzip=True)

print(f'Dataset downloaded and extracted to: {path_to_download}')


Dataset downloaded and extracted to: birds


In [4]:
# Import Tensor Flow
import tensorflow
# Import Keras
from tensorflow import keras
# Import Numpy
import numpy as np
# Import Matplotlib
import matplotlib.pyplot as plt

# Load the bird identification dataset from Kaggle https://www.kaggle.com/datasets/gpiosenka/100-bird-species
# Load the dataset
train_data = keras.utils.image_dataset_from_directory('birds/train', image_size=(224, 224), batch_size=32)
test_data = keras.utils.image_dataset_from_directory('birds/test', image_size=(224, 224), batch_size=32)
validation_data = keras.utils.image_dataset_from_directory('birds/valid', image_size=(224, 224), batch_size=32)
# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_data)
# Print the shape of the test data
print(test_data)
# Print the shape of the validation data
print(validation_data)


# Example of normalization for image data
train_data = train_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))
validation_data = validation_data.map(lambda x, y: (x / 255.0, y))

# Create a CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224, 224, 3)),
    #keras.layers.MaxPooling2D(),
    #keras.layers.Conv2D(32, 3, activation='relu'),
    #keras.layers.MaxPooling2D(),
    #keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(250, activation='softmax')
])
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Fit the model
model.fit(train_data, epochs=10, validation_data=validation_data)

# Evaluate the model
model.evaluate(test_data)

# Save the model
model.save('birds_model')

# Load the model
loaded_model = keras.models.load_model('birds_model')

# Make predictions
predictions = loaded_model.predict(test_data)
# Print the predictionsf
print(predictions)


Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
['train/ABBOTTS BABBLER/001.jpg' 'train/ABBOTTS BABBLER/007.jpg'
 'train/ABBOTTS BABBLER/008.jpg' ... 'valid/BLACK BREASTED PUFFBIRD/1.jpg'
 'valid/BLACK BREASTED PUFFBIRD/2.jpg'
 'valid/BLACK BREASTED PUFFBIRD/5.jpg']
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Epoch 1/10
2645/2645 [==============================] - 101s 38ms/step - loss: nan - accuracy: 0.0019 - val_loss: nan - val_accuracy: 0.0019
Epoch 2/10
2645/2645 [==============================] 

In [2]:
# Inspect the label format for the training data
for images, labels in train_data.take(1):  # Take 1 batch from the dataset
    print("Label examples from training data:", labels.numpy())  # Print the labels of the batch

# Inspect the label format for the test data
for images, labels in test_data.take(1):
    print("Label examples from test data:", labels.numpy())

# Inspect the label format for the validation data
for images, labels in validation_data.take(1):
    print("Label examples from validation data:", labels.numpy())


Label examples from training data: [195 424  55 101 461 462  50 313 482  15  18 198 472 238 173 250 371 496
 429 305 290 443 220 507 349 177  92 248 213 119 302 338]
Label examples from test data: [226 341 410 175 363 380 262 224 272 415 100 285  54 261 131 280 184 264
 136 520 333 253 160 142 384 510 321 233 125 301 190 194]
Label examples from validation data: [ 85 446 348 336   4 293 222 292 126 340 236  99  15 163 128 313  82 409
 483  15  50 448 107 391 322 273  19 470 250 513 154 122]


In [3]:
# Example of normalization for image data
train_data = train_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))
validation_data = validation_data.map(lambda x, y: (x / 255.0, y))
